In [1]:
import pandas as pd
import os
cases_train= pd.read_csv('data/cases_2021_train_processed_2.csv')
cases_test= pd.read_csv('data/cases_2021_test_processed_unlabelled_2.csv')


# Pre-Processing

## Selecting Features 

In [2]:
cases_train = cases_train.drop(columns='date_confirmation')
cases_train = cases_train.drop(columns='province')
cases_train.to_csv('data/ctrain.csv',index=False)
cases_test = cases_test.drop(columns='date_confirmation')
cases_test = cases_test.drop(columns='province')
cases_test.to_csv('data/ctest.csv',index=False)

## Balancing Data

In [31]:
#%pip install imbalanced-learn
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
train= pd.read_csv('data/ctrain.csv')
test= pd.read_csv('data/ctest.csv')

Total Samples Before Balancing:  17212 Distribution of Classes: Counter({'hospitalized': 13241, 'nonhospitalized': 2974, 'deceased': 997})


# Encoding

In [18]:
enc=OrdinalEncoder()
train['sex']=enc.fit_transform(train[['sex']])
test['sex']=enc.fit_transform(test[['sex']])
countries_train=enc.fit_transform(train[['country']])
train['outcome_group']=enc.fit_transform(train[['outcome_group']])
train['chronic_disease_binary']=enc.fit_transform(train[['chronic_disease_binary']])
test['chronic_disease_binary']=enc.fit_transform(test[['chronic_disease_binary']])

In [20]:
allcountries=np.append(train['country'],test['country'])
data=pd.DataFrame(allcountries, columns=['country'])
data['encoding']=enc.fit_transform(data[['country']])
data['encoding'].astype('str')
data=data.drop_duplicates()

In [23]:
for i in range(39):
    test.loc[test['country']==data['country'].iloc[i],'country']=data['encoding'].iloc[i]
    train.loc[train['country']==data['country'].iloc[i],'country']=data['encoding'].iloc[i]

# Balancing

In [ ]:
labels = train.loc[:,train.columns == 'outcome_group']
features = train.loc[:,train.columns != 'outcome_group']
counter = Counter(labels['outcome_group'])
print("Total Samples Before Balancing: ",len(train), "Distribution of Classes:", counter)
over = RandomOverSampler({2.0:5680 ,0.0:5680})
under = RandomUnderSampler({1.0:5680})
features,labels = under.fit_resample(features,labels)
features,labels = over.fit_resample(features,labels)
counter = Counter(labels['outcome_group'])
features['outcome_group'] = labels['outcome_group']
train = features
print("Total Samples After Balancing: ",len(train), "Distribution of Classes:", counter)

In [ ]:
train.to_csv('data/train.csv',index=False)
test.to_csv('data/test.csv',index=False)